# Data handling with Dataframe

Kotlin [DataFrame](https://github.com/Kotlin/dataframe) was largely inspired by pandas structures, but despite this, it has his own characteristics that make Data Analysis very understandable and concise. Most of it's readability comes from functional languages chains of transformations (the data pipeline), and the use of DSL that makes it's syntax closer to natural language.

## Overview

Dataframe is built-in kotlin jupyter kernel, so the magic command `%use` will the whole library for us

In [22]:
%use dataframe

For creating a dataframe from scratch, we can follow several approaches, and the most used are:

1. Creating `Column` objects for storing data, and assign columns to a `DataFrame`

In [77]:
val names by columnOf("foo", "bar", "baz")
val numbers by columnOf(1, 2, 3)

val df = dataFrameOf(names, numbers)
df

DataFrame: rowsCount = 3, columnsCount = 2

2. Providing a series of `Pair<String, Any>` or a `Map<String, Any>`

In [24]:
val df = dataFrameOf(
    "names" to listOf("foo", "bar", "baz"),
    "numbers" to listOf(1, 2, 3)
)

df

DataFrame: rowsCount = 3, columnsCount = 2

3. Providing an `Iterable<Column>`

````{margin}
```{note}
For more, please refer to DataFrame [constructions methods](https://kotlin.github.io/dataframe/createdataframe.html)
```
````

In [25]:
val values = (1..5).map { List(10) { x -> (x - it) * 10 }.toColumn("$it") }

dataFrameOf(values)

DataFrame: rowsCount = 10, columnsCount = 5

We can load a dataset, and compute some statics

In [26]:
val df = DataFrame.readCSV(
            "../resources/example-datasets/datasets/stock_px.csv",
            header = listOf("date", "AAPL", "MSFT", "XOM", "SPX"),
            skipLines = 1
        )
df.head(5)

DataFrame: rowsCount = 5, columnsCount = 5

In [27]:
df.groupBy { date.map { it.month }}.mean() // mean for each month, for each stock

DataFrame: rowsCount = 12, columnsCount = 5

In [28]:
val yearMeans = df.groupBy { date.map { it.year} }.mean() // compute the means for each year
    // mappig each stock and it's value to two separate columns
    .gather { AAPL and MSFT and XOM and SPX}.into("stock", "value") 
    .rename { date }.into("year")
yearMeans.head(10)

DataFrame: rowsCount = 10, columnsCount = 3

We can then **plot** those statistics with `lets-plot` library:

In [65]:
%use lets-plot

In [76]:
val p1 = ggplot(yearMeans.toMap()) { x="year" ; y="value" ; color="stock" } +
    geomLine(stat = Stat.identity, position = positionDodge(0.5), alpha = 0.7) +
    geomPoint(size=3.0, shape = 5) +
    scaleYLog10()
    
val p2 = ggplot(yearMeans.toMap()) { x="stock" ; y="value" } +
    geomBoxplot() { fill = "stock"} +
    scaleYLog10()
    

gggrid(listOf(p1, p2))


In [75]:
ggplot(yearMeans.filter { stock == "AAPL"}.toMap() )+
    geomLine(stat = Stat.identity) { x="year" ; y="value" } +
    geomPoint(color="red", size=3.5, shape = 2) { x="year" ; y="value" } +
    ylab("Value ($)") + 
    xlab("Year") +
    ggtitle("Apple Inc. Stock Price (2003-2011)")
    

## DataFrame structure

The DataFrame library, defines the following data abstractions:
- `DataColumn`: is a named, typed and ordered collection of elements.
- `DataFrame`: consists of one or several `DataColumns` with unque names and equal size.
- `DataRow` is a single row of a `DataFrame` and provides a single value for every `DataColumn`.

Because we are dealing with structured data, Dataframe provides **hierarchical** data structures using two special types of columns:
- `ColumnGroup` is a group of columns
- `FrameColumn` is a column of dataframes

This makes easy the creation of tree structures among data (very handy when reading `JSON`s files).


By nature, data frames are dynamic objects, column labels depend on the input source and also new columns could be added or deleted while wrangling.
Kotlin in contrast, is a statically typed language and all types are defined and verified ahead of execution.

For this reason, the Kotlin DataFrame library provide four different ways to access columns:
- String API
- Columns Accessors API
- KProperties API
- Extension Properties API

For detailed usage, refer to the official documentation of [column accessors](https://kotlin.github.io/dataframe/apilevels.html#list-of-access-apis).

The string API is the simplest and **unsafest** of them all. The main advantage is that it can be used at any time, including when accessing new columns in chain calls, so that this call can be made:
```kotlin
df.add("age") { ... }
    .sum("age")
```

If you're not working in an Jupyter Notebook, *Column Accessor API* provide type-safe access to columns, but does not ensure that the columns really exist in a particular dataframe. Similarly, when working in an IDE, *KProperties API* is useful when you've already declared classes in you application business logic with fields that correspond columns of a DataFrame.

Otherwise, if you're working inside a notebook, you can use Extension Properties API, which are the safest and convenient to use, with the trade-off of execution speed in the moment of generation.

### `DataColumn`

Every `DataColumn` object has a unique type (even nullable values like `String?`) and several data mapped into rows.

As pointed out above, we can create a column object with the `by` keyword

In [27]:
val col by columnOf("a", "b", "c")

Following this approach, the name of the column is the name we gave to the variable, and we can use the *column accessor* API for better type safety.

Similarly, we can explicitly cast the column to a `Ktype`, and the result will be a `ColumnAccessor`:

In [283]:
val col by column<Double>("values")

With the `ColumnAccessor`, we can convert it to a `DataColumn` using `withValues` function:

In [288]:
val age by column<Int>()
val ageCol1 = age.withValues(15, 20)
val ageCol2 = age.withValues(10..20)

`List` and `Set` from the standard library have an *extension function* that can convert the collection to a column with the provided name.

In [282]:
val col = List(5) { it * 2 }.toColumn("values")

A column can be of three types:
- `ValueColumn`: stores primitives data (by now, the underlying structure is a `List`)
- `ColumnGroup`: stores nested columns
    - For referencing a nested column we can use

```kotlin
val name by columnGroup()
val firstName by name.column<String>()
```

- `FrameColumn`: stores a nested `DataFrame`

### `DataFrame`

A `DataFrame` represent a list of `DataColumn`s. Columns in a `DataFrame` must have **equal size** and **names**.

The simplest way to create a `DataFrame` is using the function `dataFrameOf`

In [289]:
val df = dataFrameOf("name", "age")(
    "Alice", 15,
    "Bob", 20,
    "Charlie", 25
)
df

DataFrame: rowsCount = 3, columnsCount = 2

For all the methods for building a dataframe, see the [official documentation](https://kotlin.github.io/dataframe/createdataframe.html)

In the following sections we will see most of the operations that could be made on top of `DataFrame`.

```{note}
Unlike `pandas`, kotlin `Dataframe` does not provide explicit indices.
```

## Operations Overview

As said before, data transformations pipelines are designed in functional style so that the whole processing can be represented as a sequential chain of operations.
`DataFrame`s are immutable, and every operations return a copy of the object instance *reusing* underlying data structures as much as possible.

**Operations** can be divided in three categories:
- **General Operations**: all basic operations that can be called on a dataframe (e.g. `schema()`, `sum()`, `move()`, `map()`, `filter()`, ...)
- **Multiplex Operations**: more complex operations that does not return a new `DataFrame` immediately, instead they provide an intermediate object that is used for further configurations. Every multiplex operation follows the schema:
    1. Use a column selector to select target columns
    2. Additional configuration functions
    3. Terminal function that returns the modified `DataFrame`
    - Most of these operations end with `into` or `with`, and the following convention is used:
        - `into` defines column names for storing the result.
        - `where` defines row-wise data transformation.
- **Shortcut Operations**: shortcut for more general operations (e.g. `rename` is a special case for `move`, `fillNA` is a special case for `update`, ...)

### Essential Functionalities

This section will walk you through the fundamental mechanics of interacting with the data contained in `DataFrame`s.

#### Indexing, Selection and Filtering

In [309]:
val obj = dataFrameOf(
    "letters" to listOf("a", "b", "c", "d"),
    "nums" to listOf(0.0, 1.0, 2.0, 3.0)
)
obj

DataFrame: rowsCount = 4, columnsCount = 2

We can access by row with

In [310]:
obj[1]

DataRow: index = 1, columnsCount = 2

And we can access by columns with:

In [313]:
obj["nums"]

DataColumn: name = "nums", type = Double, size = 4

We can select multiple columns with the usual notation:

In [326]:
obj["nums", "letters"]

DataFrame: rowsCount = 4, columnsCount = 2

We can select also ranges (note that the second boundary of the range is *included*)

In [324]:
obj[0..2] // obj[0 until 3]

DataFrame: rowsCount = 3, columnsCount = 2

The `[...]`  operator calls the `get()` method, so:
```kotlin
obj[0] == obj.get(0) // true
obj["nums"] == obj.getColumn(1) == obj.getColumn("nums") // true
```

Unlike python, kotlin does not provide filtering inside square brackets, but it offers the filter method that can be more understandable

In [350]:
obj.filter { nums.toInt() % 2 == 0 }

DataFrame: rowsCount = 2, columnsCount = 2

instead of python's:
```python
obj[(obj["nums"] % 2 == 0)]
```

#### Arithmetic Operations

Unlike pandas, operations between dataframes are not defined by default.